In [37]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'


In [38]:
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [39]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json["events"])


In [40]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [41]:
slim_elements_df = elements_df[['second_name','first_name','team','element_type','selected_by_percent',
       'minutes','value_season','total_points','bps', 
       'influence', 'creativity', 'threat','goals_scored','assists','clean_sheets', 'goals_conceded', 
       'own_goals','penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards','saves', 'bonus']]

In [42]:
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

C:\Users\steve\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [43]:
slim_elements_df.head()

,second_name,first_name,team,element_type,selected_by_percent,minutes,value_season,total_points,bps,influence,...,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,position
0,Leno,Bernd,1,1,2.3,3131,26.2,131,625,702.2,...,11,37,1,1,0,0,1,86,11,Goalkeeper
1,Rúnarsson,Rúnar Alex,1,1,1.2,15,0.2,1,8,16.6,...,0,0,0,0,0,0,0,2,0,Goalkeeper
2,Borges Da Silva,Willian,1,3,0.3,1398,12.0,78,306,256.2,...,3,17,0,0,0,0,0,0,9,Midfielder
3,Aubameyang,Pierre-Emerick,1,4,3.9,2330,13.1,131,358,493.8,...,10,26,1,0,0,2,0,0,11,Forward
4,Soares,Cédric,1,2,0.3,744,6.2,28,125,110.8,...,2,11,0,0,0,1,0,0,3,Defender


In [44]:
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)
slim_elements_df['value_season'] = slim_elements_df.value_season.astype(float)

C:\Users\steve\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [45]:
slim_elements_df = slim_elements_df.sort_values('value_season', ascending = False)


In [46]:
slim_elements_df = slim_elements_df.loc[slim_elements_df["minutes"] >= 10]
try:
    slim_elements_df["Avg Playtime"] = slim_elements_df['minutes'] / 38
except:
    0
try:
    slim_elements_df["PPG"] = slim_elements_df['total_points'] / 38
except:
    0
try:
    slim_elements_df["PPGP"] = slim_elements_df["total_points"] / (slim_elements_df["minutes"]/90)
except:
    0
slim_elements_df.corr()

,element_type,minutes,value_season,total_points,bps,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,Avg Playtime,PPG,PPGP
element_type,1.000000,-0.078709,-0.145758,0.077929,-0.149380,0.483638,0.382272,-0.059708,-0.059496,-0.160142,-0.343959,0.181288,0.017215,-0.025158,-0.475669,0.113346,-0.078709,0.077929,0.378694
minutes,-0.078709,1.000000,0.931864,0.859784,0.936159,0.345160,0.412607,0.875806,0.905842,0.166698,0.211677,0.131003,0.499740,0.182955,0.281364,0.604238,1.000000,0.859784,-0.088064
value_season,-0.145758,0.931864,1.000000,0.890987,0.944179,0.346389,0.386808,0.866091,0.802630,0.167660,0.283944,0.124646,0.358292,0.167529,0.367685,0.684399,0.931864,0.890987,0.075847
total_points,0.077929,0.859784,0.890987,1.000000,0.916530,0.676027,0.656049,0.843243,0.698312,0.087852,0.197166,0.242018,0.252983,0.106628,0.240307,0.853413,0.859784,1.000000,0.280624
bps,-0.149380,0.936159,0.944179,0.916530,1.000000,0.403823,0.467300,0.881539,0.785500,0.151187,0.250679,0.183001,0.386749,0.153421,0.323539,0.746704,0.936159,0.916530,0.075822
goals_scored,0.483638,0.345160,0.346389,0.676027,0.403823,1.000000,0.629386,0.343823,0.274598,-0.050272,-0.110721,0.349470,0.008925,0.038667,-0.156180,0.708599,0.345160,0.676027,0.465796
assists,0.382272,0.412607,0.386808,0.656049,0.467300,0.629386,1.000000,0.420824,0.321662,-0.077890,-0.131430,0.249638,0.099223,-0.034157,-0.186496,0.615531,0.412607,0.656049,0.348820
clean_sheets,-0.059708,0.875806,0.866091,0.843243,0.881539,0.343823,0.420824,1.000000,0.635573,0.128881,0.188452,0.193231,0.422529,0.175548,0.215562,0.588821,0.875806,0.843243,0.060658
goals_conceded,-0.059496,0.905842,0.802630,0.698312,0.785500,0.274598,0.321662,0.635573,1.000000,0.134282,0.179766,0.077673,0.486081,0.168847,0.268239,0.482650,0.905842,0.698312,-0.203635
own_goals,-0.160142,0.166698,0.167660,0.087852,0.151187,-0.050272,-0.077890,0.128881,0.134282,1.000000,0.056082,-0.065590,0.068677,0.146620,0.063018,0.069719,0.166698,0.087852,-0.098662


In [51]:
slim_elements_df["Full Name"] = slim_elements_df["second_name"] + ", " + slim_elements_df["first_name"] + " - " + slim_elements_df["team"]

In [52]:
slim_elements_df.to_csv('Player Data.csv')

In [ ]:
#Goalies:
